<a href="https://colab.research.google.com/github/linneasandersen/dalas-project/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Get data
- Wikipedia
- OEC
- FAO
2.   Store data in pandas
3. Inspect




In [16]:
import requests
import pandas as pd
from pathlib import Path
import bs4
import lxml
import pandas
import urllib

from urllib import request

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# OEC

In [5]:
# Define the years to retrieve data for
years = range(1995, 2024)

In [6]:
# Iterate and fetch export data
df_exports = []
df_imports = []
url = "https://api-v2.oec.world/tesseract/data.jsonrecords"

for year in years:
    params_export = {
        "cube": "trade_i_baci_a_92",
        "locale": "en",
        "drilldowns": "Exporter Country,Year,HS4",
        "measures": "Trade Value",
        "include": f"Year:{year}"
    }
    r = requests.get(url, params=params_export)
    try:
        df = pd.DataFrame(r.json()["data"])
        df_exports.append(df)
    except KeyError:
        print(f"No data found for export in {year}")

    params_import = {
    "cube": "trade_i_baci_a_92",
    "locale": "en",
    "drilldowns": "Importer Country,Year,HS4",
    "measures": "Trade Value",
    "include": f"Year:{year}"
    }
    r = requests.get(url, params=params_import)
    try:
        df = pd.DataFrame(r.json()["data"])
        df_imports.append(df)
    except KeyError:
        print(f"No data found for import in {year}")


# concatenate
df_exports = pd.concat(df_exports)
df_imports = pd.concat(df_imports)

print("Finished fetching export data.")

Finished fetching export data.


In [7]:
# display data
display(df_exports)
display(df_imports)
print(df_exports.columns)
print(df_imports.columns)


# more possibilities for columns, see cubeOECinfo.json

,Exporter Country ID,Exporter Country,HS4 ID,HS4,Year,Trade Value
0,afago,Angola,10302,Non-fillet Fresh Fish,1995,35266.0
1,afago,Angola,10303,Non-fillet Frozen Fish,1995,7956102.0
2,afago,Angola,10304,Fish Fillets,1995,278508.0
3,afago,Angola,10305,Dried/Salted/Smoked/Brined Fish,1995,3682.0
4,afago,Angola,10306,Crustaceans,1995,21217946.0
...,...,...,...,...,...,...
175624,saven,Venezuela,219702,Prints,2023,1327961.0
175625,saven,Venezuela,219703,Sculptures,2023,8853599.0
175626,saven,Venezuela,219704,Revenue Stamps,2023,147449.0
175627,saven,Venezuela,219705,Collector's Items,2023,158160.0


,HS4 ID,HS4,Importer Country ID,Importer Country,Year,Trade Value
0,10101,Horses,afbfa,Burkina Faso,1995,1351.0
1,10101,Horses,afciv,Cote d'Ivoire,1995,9136.0
2,10101,Horses,afcod,Democratic Republic of the Congo,1995,1811.0
3,10101,Horses,afcpv,Cape Verde,1995,567.0
4,10101,Horses,afdza,Algeria,1995,30570.0
...,...,...,...,...,...,...
233276,219706,Antiques,saecu,Ecuador,2023,3894.0
233277,219706,Antiques,saguy,Guyana,2023,14955.0
233278,219706,Antiques,saper,Peru,2023,335766.0
233279,219706,Antiques,sapry,Paraguay,2023,61105.0


Index(['Exporter Country ID', 'Exporter Country', 'HS4 ID', 'HS4', 'Year',
       'Trade Value'],
      dtype='object')
Index(['HS4 ID', 'HS4', 'Importer Country ID', 'Importer Country', 'Year',
       'Trade Value'],
      dtype='object')


Data set size is order of magnitude 10^6 for both import and export
Currently 6 columns, but more possibilities

## Harmonized system of products

In [8]:
data_dir = Path("/content/drive/MyDrive/DALAS/data/raw")
data_path_products = data_dir / "hs_product_classification.csv"

In [9]:
print(data_path_products)
df = pd.read_csv(
    data_path_products,
    sep=';',        # semicolon separator
    quotechar='"'
)
print(df.head())
print(df.columns)

/content/drive/MyDrive/DALAS/data/raw/hs_product_classification.csv
    OEC ID     HS ID Product Level  \
0        1         1       Section   
1      101        01           HS2   
2    10101     01.01           HS4   
3  1010110  01.01.10           HS6   
4  1010111  01.01.11           HS6   

                                        Product Name  \
0                                    Animal Products   
1                                       Live animals   
2                                             Horses   
3  Live horses/asses/mules/hinnies: pure-bred bre...   
4                          Pure-bred Breeding Horses   

                             Revision  
0  HS92,HS96,HS02,HS07,HS12,HS17,HS22  
1  HS92,HS96,HS02,HS07,HS12,HS17,HS22  
2  HS92,HS96,HS02,HS07,HS12,HS17,HS22  
3                           HS02,HS07  
4                           HS92,HS96  
Index(['OEC ID', 'HS ID', 'Product Level', 'Product Name', 'Revision'], dtype='object')


# FAO


In [10]:
data_dir = Path("/content/drive/MyDrive/DALAS/data/raw")
zip_file = data_dir / "Environment_Temperature_change_E_All_Data.zip"

In [11]:
from pathlib import Path
from zipfile import ZipFile

extract_dir = data_dir / "extracted"
extract_dir.mkdir(parents=True, exist_ok=True)

# Extract all files
with ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


Files extracted to /content/drive/MyDrive/DALAS/data/raw/extracted


In [12]:
# Find extracted Excel files
files = list(extract_dir.glob("*.csv"))

for file in files:
    print(file.name)

Environment_Temperature_change_E_AreaCodes.csv
Environment_Temperature_change_E_Elements.csv
Environment_Temperature_change_E_Flags.csv
Environment_Temperature_change_E_All_Data.csv
Environment_Temperature_change_E_All_Data_NOFLAG.csv


In [13]:
# print columns of all datasets, size of table
for file in files:
    print(file.name)
    df = pd.read_csv(file)
    print(df.shape)
    print(df.columns)
    print("---------------------------------------------------------------------------------")


Environment_Temperature_change_E_AreaCodes.csv
(284, 3)
Index([' Area Code', ' M49 Code', ' Area'], dtype='object')
---------------------------------------------------------------------------------
Environment_Temperature_change_E_Elements.csv
(2, 2)
Index([' Element Code', ' Element'], dtype='object')
---------------------------------------------------------------------------------
Environment_Temperature_change_E_Flags.csv
(2, 2)
Index([' Flag', ' Description'], dtype='object')
---------------------------------------------------------------------------------
Environment_Temperature_change_E_All_Data.csv
(9656, 136)
Index(['Area Code', 'Area Code (M49)', 'Area', 'Months Code', 'Months',
       'Element Code', 'Element', 'Unit', 'Y1961', 'Y1961F',
       ...
       'Y2020', 'Y2020F', 'Y2021', 'Y2021F', 'Y2022', 'Y2022F', 'Y2023',
       'Y2023F', 'Y2024', 'Y2024F'],
      dtype='object', length=136)
---------------------------------------------------------------------------------
Envir

In [14]:
# Load first file
file = files[0]
print(file)
df = pd.read_csv(file)
print(df.head())

/content/drive/MyDrive/DALAS/data/raw/extracted/Environment_Temperature_change_E_AreaCodes.csv
   Area Code M49 Code            Area
0          2     '004     Afghanistan
1       5100     '002          Africa
2          3     '008         Albania
3          4     '012         Algeria
4          5     '016  American Samoa


9656 rows, 136 columns

# Wikipedia



In [18]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_tariff_rate"
r = request.Request(url, headers={"User-Agent": "Mozilla/5.0"})

request_text = request.urlopen(r).read()
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

<title>List of countries by tariff rate - Wikipedia</title>


In [47]:
tableau_participants = page.find('table', {'class' : 'wikitable'})
table_body = tableau_participants.find('tbody')
rows = table_body.find_all('tr')

header_row1 = rows[0].find_all(['th', 'td'])
header_row2 = rows[1].find_all(['th', 'td'])

header1_text = [cell.get_text(strip=True) for cell in header_row1]
header2_text = [cell.get_text(strip=True) for cell in header_row2]

combined_headers = [header1_text[0]]
combined_headers.append(f"{header1_text[1]} - {header2_text[0]}")
combined_headers.append(f"{header1_text[1]} - {header2_text[1]}")
combined_headers.append(f"{header1_text[2]} - {header2_text[2]}")
combined_headers.append(f"{header1_text[2]} - {header2_text[3]}")
combined_headers.append(f"{header1_text[3]} - {header2_text[4]}")
combined_headers.append(f"{header1_text[3]} - {header2_text[5]}")

data_rows = rows[2:]

results = []
expected_cols = len(combined_headers)

for row in data_rows:
    cols = row.find_all(['th', 'td'])
    cols = [ele.text.strip() for ele in cols]

    if len(cols) > 3 and cols[3] == "EU member":
        if len(cols) >= 6:
            corrected_cols = cols[:4]
            corrected_cols.append(None)
            corrected_cols.append(cols[4])
            corrected_cols.append(cols[5] if len(cols) > 5 else None)
            cols = corrected_cols
        else:
             print(f"Warning: Row with 'EU member' has unexpected structure: {cols}")
             while len(cols) < expected_cols:
                cols.append(None)


    while len(cols) < expected_cols:
        cols.append(None)
    if len(cols) > expected_cols:
         print(f"Warning: Row still has more than {expected_cols} columns after processing: {cols}")
         cols = cols[:expected_cols]


    results.append(cols)

df = pd.DataFrame(data=results, columns=combined_headers)

display(df.head())
display(df[df['WTO[2][6] - Simple average applied MFN tariff, all products'] == 'EU member'].head())

,Country/Territory/Region/Group,"WB[3] - Tariff rate, applied, weighted mean, all products",WB[3] - Year,"WTO[2][6] - Simple average applied MFN tariff, all products",WTO[2][6] - Year,UNCTAD[1] - Import tariff rates on non-agricultural and non-fuel products,UNCTAD[1] - Year
0,WORLD,2.59%,2017,,,,
1,Aruba,0%,2021,,,10.80%,2021
2,Afghanistan,0%,2018,6.5%,2018,0%,2018
3,Albania,0%,2021,3.6%,2021,0%,2021
4,Algeria,,,18.9%,2022,,


,Country/Territory/Region/Group,"WB[3] - Tariff rate, applied, weighted mean, all products",WB[3] - Year,"WTO[2][6] - Simple average applied MFN tariff, all products",WTO[2][6] - Year,UNCTAD[1] - Import tariff rates on non-agricultural and non-fuel products,UNCTAD[1] - Year
12,Austria,1.39%,2021,EU member,None,0.96%,2021
18,Belgium,1.39%,2021,EU member,None,1.33%,2021
23,Bulgaria,1.39%,2021,EU member,None,1.28%,2019
48,Cyprus,1.39%,2021,EU member,None,1.46%,2021
49,Czech Republic,1.39%,2021,EU member,None,1.14%,2021


In [49]:
# clean headers to remove references, i.e [1]
df.columns = df.columns.str.replace(r'\[\d+\]', '', regex=True)
display(df.head())

,Country/Territory/Region/Group,"WB - Tariff rate, applied, weighted mean, all products",WB - Year,"WTO - Simple average applied MFN tariff, all products",WTO - Year,UNCTAD - Import tariff rates on non-agricultural and non-fuel products,UNCTAD - Year
0,WORLD,2.59%,2017,,,,
1,Aruba,0%,2021,,,10.80%,2021
2,Afghanistan,0%,2018,6.5%,2018,0%,2018
3,Albania,0%,2021,3.6%,2021,0%,2021
4,Algeria,,,18.9%,2022,,


https://docs.google.com/spreadsheets/d/1jeHGiYRuVt_zJg12dD5b8BtEh3qNi1p8L-9tPK56zak/edit#gid=0


In [55]:
# Separate dataframes into 3
df_WB = df.iloc[:, :3].copy()

# col 1, 4, 5
df_WTO = df.iloc[:, [0, 3, 4]].copy()

# col 1, 6, 7
df_UNCTAD = df.iloc[:, [0, 5, 6]].copy()

# Replace empty strings with NaN to be recognized by dropna
df_WB.replace('', pd.NA, inplace=True)
df_WTO.replace('', pd.NA, inplace=True)
df_UNCTAD.replace('', pd.NA, inplace=True)

# remove rows where 2nd or 3rd col is empty
df_WB = df_WB.dropna(subset=[df_WB.columns[1], df_WB.columns[2]])
df_WTO = df_WTO.dropna(subset=[df_WTO.columns[1], df_WTO.columns[2]])
df_UNCTAD = df_UNCTAD.dropna(subset=[df_UNCTAD.columns[1], df_UNCTAD.columns[2]])

# shapes
print(df_WB.shape)
print(df_WTO.shape)
print(df_UNCTAD.shape)

# Display the new DataFrames
display(df_WB.head())
display(df_WTO.head())
display(df_UNCTAD.head())

(194, 3)
(157, 3)
(195, 3)


,Country/Territory/Region/Group,"WB - Tariff rate, applied, weighted mean, all products",WB - Year
0,WORLD,2.59%,2017
1,Aruba,0%,2021
2,Afghanistan,0%,2018
3,Albania,0%,2021
5,Angola,0%,2021


,Country/Territory/Region/Group,"WTO - Simple average applied MFN tariff, all products",WTO - Year
2,Afghanistan,6.5%,2018
3,Albania,3.6%,2021
4,Algeria,18.9%,2022
5,Angola,11%,2023
7,Antigua and Barbuda,9.9%,2021


,Country/Territory/Region/Group,UNCTAD - Import tariff rates on non-agricultural and non-fuel products,UNCTAD - Year
1,Aruba,10.80%,2021
2,Afghanistan,0%,2018
3,Albania,0%,2021
5,Angola,0%,2021
6,Anguilla,13.14%,2021


In [56]:
# what years are covered
print(df['Year'].unique())



KeyError: 'Year'